In [1]:
#Refernce tutorial https://www.datacamp.com/community/tutorials/apache-spark-tutorial-machine-learning
import findspark
findspark.init("/home/kyesh/Programs/spark-2.3.0-bin-hadoop2.7/")

In [2]:
# Import SparkSession
from pyspark.sql import SparkSession

# Build the SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("Linear Regression Model") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
   
sc = spark.sparkContext

In [5]:
# Load in the data
#rdd = sc.textFile('/home/kyesh/Downloads/titanic/train.csv')

from pyspark.sql import SQLContext
sql_context = SQLContext(sc)
df = sql_context.read.csv('/home/kyesh/Downloads/titanic/train.csv', header=True, inferSchema=True)
df.printSchema()

df.count()

# Load in the header
#header = sc.textFile('/home/kyesh/Downloads/titanic/trainHeader.csv')

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



791

In [4]:
# Show the top 20 rows 
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [10]:
df.select(df['Pclass'],df["name1"],df["name2"],df['Sex']).show(508)

+------+--------------------+--------------------+------+
|Pclass|               name1|               name2|   Sex|
+------+--------------------+--------------------+------+
|     3|             "Braund|    Mr. Owen Harris"|  male|
|     1|            "Cumings| Mrs. John Bradle...|female|
|     3|          "Heikkinen|        Miss. Laina"|female|
|     1|           "Futrelle| Mrs. Jacques Hea...|female|
|     3|              "Allen|  Mr. William Henry"|  male|
|     3|              "Moran|          Mr. James"|  male|
|     1|           "McCarthy|      Mr. Timothy J"|  male|
|     3|            "Palsson| Master. Gosta Le...|  male|
|     3|            "Johnson| Mrs. Oscar W (El...|female|
|     2|             "Nasser| Mrs. Nicholas (A...|female|
|     3|          "Sandstrom| Miss. Marguerite...|female|
|     1|            "Bonnell|    Miss. Elizabeth"|female|
|     3|        "Saundercock|  Mr. William Henry"|  male|
|     3|          "Andersson|   Mr. Anders Johan"|  male|
|     3|      

In [99]:
import pyspark.sql.functions as func

grpB = "Age"
df.groupBy(grpB).agg(func.count(func.lit(1)),func.mean("Survived")).sort("count(1)",ascending=False).show()

df.groupBy(grpB).agg(func.count(func.lit(1)),func.mean("Survived")).sort("Age",ascending=True).show()

temp = df.groupBy(grpB).agg(func.count(func.lit(1)),func.mean("Survived"))

temp.sort("avg(Survived)",ascending=False).filter(temp["count(1)"] > 2).show()

agetest = 0

df_temp = df.select(df['Age'],df['Survived']).filter(df.Age > agetest)

df_temp.filter(df.Age < agetest + 6).agg(func.count(func.lit(1)),func.mean("Survived")).show()

+----+--------+-------------------+
| Age|count(1)|      avg(Survived)|
+----+--------+-------------------+
|null|     161|0.30434782608695654|
|24.0|      28|                0.5|
|22.0|      26| 0.4230769230769231|
|18.0|      23|0.34782608695652173|
|19.0|      23|0.34782608695652173|
|21.0|      22|0.22727272727272727|
|30.0|      22| 0.4090909090909091|
|28.0|      22| 0.2727272727272727|
|36.0|      22|                0.5|
|25.0|      20|               0.25|
|29.0|      20|                0.4|
|32.0|      16|                0.5|
|35.0|      16|             0.6875|
|26.0|      15| 0.3333333333333333|
|34.0|      14|0.42857142857142855|
|16.0|      14|0.35714285714285715|
|20.0|      13|0.23076923076923078|
|23.0|      13|0.38461538461538464|
|27.0|      13| 0.5384615384615384|
|40.0|      13|0.46153846153846156|
+----+--------+-------------------+
only showing top 20 rows

+----+--------+-------------------+
| Age|count(1)|      avg(Survived)|
+----+--------+-------------------+
|n

In [39]:
# Print the data types of all `df` columns
# df.dtypes

# Print the schema of `df`
df.printSchema()

root
 |-- Age: string (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Fare: string (nullable = true)
 |-- Name1: string (nullable = true)
 |-- Name2: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- PassengerId: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Survived: string (nullable = true)
 |-- Ticket: string (nullable = true)



In [13]:
# Import all from `sql.types`
from pyspark.sql.types import *

# Write a custom function to convert the data type of DataFrame columns
def convertColumn(df, nameType):
  for name, type_ in nameType: 
     df = df.withColumn(name, df[name].cast(newType))
  return df 

# Assign all column names to `columns`
nameType = [('Age',), 
            ('Fare',),
            ('PassengerId',),
            'latitude', 'longitude', 'medianHouseValue', 'medianIncome', 'population', 'totalBedRooms', 'totalRooms']

# Conver the `df` columns to `FloatType()`
df = convertColumn(df, nameType)

ValueError: need more than 1 value to unpack

In [22]:
# Print the data types of all `df` columns
# df.dtypes

# Print the schema of `df`
df.printSchema()

root
 |-- households: float (nullable = true)
 |-- housingMedianAge: float (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- medianHouseValue: float (nullable = true)
 |-- medianIncome: float (nullable = true)
 |-- population: float (nullable = true)
 |-- totalBedRooms: float (nullable = true)
 |-- totalRooms: float (nullable = true)



In [23]:
df.select('population','totalBedRooms').show(10)

+----------+-------------+
|population|totalBedRooms|
+----------+-------------+
|     322.0|        129.0|
|    2401.0|       1106.0|
|     496.0|        190.0|
|     558.0|        235.0|
|     565.0|        280.0|
|     413.0|        213.0|
|    1094.0|        489.0|
|    1157.0|        687.0|
|    1206.0|        665.0|
|    1551.0|        707.0|
+----------+-------------+
only showing top 10 rows



In [24]:
df.groupBy("housingMedianAge").count().sort("housingMedianAge",ascending=False).show()

+----------------+-----+
|housingMedianAge|count|
+----------------+-----+
|            52.0| 1273|
|            51.0|   48|
|            50.0|  136|
|            49.0|  134|
|            48.0|  177|
|            47.0|  198|
|            46.0|  245|
|            45.0|  294|
|            44.0|  356|
|            43.0|  353|
|            42.0|  368|
|            41.0|  296|
|            40.0|  304|
|            39.0|  369|
|            38.0|  394|
|            37.0|  537|
|            36.0|  862|
|            35.0|  824|
|            34.0|  689|
|            33.0|  615|
+----------------+-----+
only showing top 20 rows



In [25]:
df.describe().show()

+-------+-----------------+------------------+-----------------+-------------------+------------------+------------------+------------------+-----------------+------------------+
|summary|       households|  housingMedianAge|         latitude|          longitude|  medianHouseValue|      medianIncome|        population|    totalBedRooms|        totalRooms|
+-------+-----------------+------------------+-----------------+-------------------+------------------+------------------+------------------+-----------------+------------------+
|  count|            20640|             20640|            20640|              20640|             20640|             20640|             20640|            20640|             20640|
|   mean|499.5396802325581|28.639486434108527|35.63186143109965|-119.56970444871473|206855.81690891474|3.8706710030346416|1425.4767441860465|537.8980135658915|2635.7630813953488|
| stddev|382.3297528316098| 12.58555761211163|2.135952380602968|  2.003531742932898|115395.61587441359|1.

In [26]:
# Import all from `sql.functions` 
from pyspark.sql.functions import *

# Adjust the values of `medianHouseValue`
df = df.withColumn("medianHouseValue", col("medianHouseValue")/100000)

# Show the first 2 lines of `df`
df.take(2)

[Row(households=126.0, housingMedianAge=41.0, latitude=37.880001068115234, longitude=-122.2300033569336, medianHouseValue=4.526, medianIncome=8.325200080871582, population=322.0, totalBedRooms=129.0, totalRooms=880.0),
 Row(households=1138.0, housingMedianAge=21.0, latitude=37.86000061035156, longitude=-122.22000122070312, medianHouseValue=3.585, medianIncome=8.301400184631348, population=2401.0, totalBedRooms=1106.0, totalRooms=7099.0)]

In [27]:
# Import all from `sql.functions` if you haven't yet
from pyspark.sql.functions import *

# Divide `totalRooms` by `households`
roomsPerHousehold = df.select(col("totalRooms")/col("households"))

# Divide `population` by `households`
populationPerHousehold = df.select(col("population")/col("households"))

# Divide `totalBedRooms` by `totalRooms`
bedroomsPerRoom = df.select(col("totalBedRooms")/col("totalRooms"))

# Add the new columns to `df`
df = df.withColumn("roomsPerHousehold", col("totalRooms")/col("households")) \
   .withColumn("populationPerHousehold", col("population")/col("households")) \
   .withColumn("bedroomsPerRoom", col("totalBedRooms")/col("totalRooms"))
   
# Inspect the result
df.first()

Row(households=126.0, housingMedianAge=41.0, latitude=37.880001068115234, longitude=-122.2300033569336, medianHouseValue=4.526, medianIncome=8.325200080871582, population=322.0, totalBedRooms=129.0, totalRooms=880.0, roomsPerHousehold=6.984126984126984, populationPerHousehold=2.5555555555555554, bedroomsPerRoom=0.14659090909090908)

In [28]:
# Re-order and select columns
df = df.select("medianHouseValue", 
              "totalBedRooms", 
              "population", 
              "households", 
              "medianIncome", 
              "roomsPerHousehold", 
              "populationPerHousehold", 
              "bedroomsPerRoom")

In [29]:
# Import `DenseVector`
from pyspark.ml.linalg import DenseVector

# Define the `input_data` 
input_data = df.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

# Replace `df` with the new DataFrame
df = spark.createDataFrame(input_data, ["label", "features"])

In [30]:
# Import `StandardScaler` 
from pyspark.ml.feature import StandardScaler

# Initialize the `standardScaler`
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

# Fit the DataFrame to the scaler
scaler = standardScaler.fit(df)

# Transform the data in `df` with the scaler
scaled_df = scaler.transform(df)

# Inspect the result
scaled_df.take(2)

[Row(label=4.526, features=DenseVector([129.0, 322.0, 126.0, 8.3252, 6.9841, 2.5556, 0.1466]), features_scaled=DenseVector([0.3062, 0.2843, 0.3296, 4.3821, 2.8228, 0.2461, 2.5264])),
 Row(label=3.585, features=DenseVector([1106.0, 2401.0, 1138.0, 8.3014, 6.2381, 2.1098, 0.1558]), features_scaled=DenseVector([2.6255, 2.1202, 2.9765, 4.3696, 2.5213, 0.2031, 2.6851]))]

In [31]:
# Split the data into train and test sets
train_data, test_data = scaled_df.randomSplit([.8,.2],seed=1234)

In [42]:
# Import `LinearRegression`
from pyspark.ml.regression import LinearRegression

# Initialize `lr`
lr = LinearRegression(labelCol="label", maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the data to the model
linearModel = lr.fit(train_data)

In [43]:
# Generate predictions
predicted = linearModel.transform(test_data)

# Extract the predictions and the "known" correct labels
predictions = predicted.select("prediction").rdd.map(lambda x: x[0])
labels = predicted.select("label").rdd.map(lambda x: x[0])

# Zip `predictions` and `labels` into a list
predictionAndLabel = predictions.zip(labels).collect()

# Print out first 5 instances of `predictionAndLabel` 
predictionAndLabel[:5]

[(0.8134519322113867, 0.14999),
 (1.3128391350327204, 0.14999),
 (1.8009833788708067, 0.14999),
 (0.8554237492441334, 0.283),
 (0.8192322857580903, 0.366)]

In [44]:
# Coefficients for the model
linearModel.coefficients

# Intercept for the model
#linearModel.intercept

DenseVector([0.0001, -0.0002, 0.0004, 0.4359, -0.0004, 0.0, 2.9623])

In [38]:
# Get the RMSE
linearModel.summary.rootMeanSquaredError

# Get the R2
linearModel.summary.r2

0.42282227755911483

In [16]:
spark.stop()
